## Load Dataset

In [6]:
from image_datasets.IXIdataset import IXIDataset
from sklearn.model_selection import train_test_split

root = "/lustre/fswork/projects/rech/krk/usy14zi/datasets/IXI-dataset/size64/"
dataset = IXIDataset(root, mode="train")

train_set, val_set = train_test_split(dataset_train, test_size=41)

channels, image_size, _ = dataset[0]['T1'].shape
print(f"Image of size {image_size}, with {channels} channel(s).")

Image of size 64, with 1 channel(s).


In [7]:
from torch.utils.data import DataLoader

batch_size = 20

train_loader = DataLoader(
    train_set,
    batch_size=batch_size,
    shuffle=False,
)
val_loader = DataLoader(
    val_set,
    batch_size=batch_size,
    shuffle=False,
)

## Initialize denoising model

In [8]:
import torch 
from model.unet import Unet
from torchinfo import summary

device = "cuda" if torch.cuda.is_available() else "cpu"

model = Unet(
    dim=image_size,
    channels=channels,
    dim_mults=(1, 2, 4,)
)
model.to(device)

summary(model)
#print(sum(p.numel() for p in model.parameters() if p.requires_grad))

Layer (type:depth-idx)                                       Param #
Unet                                                         --
├─Conv2d: 1-1                                                128
├─Sequential: 1-2                                            --
│    └─SinusoidalPositionEmbeddings: 2-1                     --
│    └─Linear: 2-2                                           16,640
│    └─GELU: 2-3                                             --
│    └─Linear: 2-4                                           65,792
├─ModuleList: 1-3                                            --
│    └─ModuleList: 2-5                                       --
│    │    └─ResnetBlock: 3-1                                 107,008
│    │    └─ResnetBlock: 3-2                                 107,008
│    │    └─Residual: 3-3                                    33,088
│    │    └─Sequential: 3-4                                  16,448
│    └─ModuleList: 2-6                                       --
│    │  

## Initialize diffusion class

In [9]:
from diffusion.time_scheduler import quadratic_beta_schedule

timesteps = 600
betas = quadratic_beta_schedule(timesteps, beta_start=0.00001, beta_end=0.01)

In [10]:
from diffusion.diffusion import DiffusionModel

ddpm = DiffusionModel(model, timesteps, betas, device, loss_type='l2')

## Train the model

In [23]:
from torch.optim import Adam

epochs = 100
optimizer = Adam(model.parameters(), lr=3e-4)

In [ ]:
ddpm.train(epochs, optimizer, train_loader, val_loader)

Epoch 0: 100%|##########| 24/24 [00:13<00:00,  1.84it/s]


Loss: 0.762143088504672


Epoch 1: 100%|##########| 24/24 [00:03<00:00,  7.85it/s]


Loss: 0.3397834428275625


Epoch 2: 100%|##########| 24/24 [00:02<00:00,  8.21it/s]


...

Epoch 61: 100%|##########| 24/24 [00:06<00:00,  3.80it/s]


Loss: 0.08875709911808372


Epoch 62: 100%|##########| 24/24 [00:06<00:00,  3.77it/s]


Loss: 0.08041661302559078


Epoch 63:  83%|########3 | 20/24 [00:05<00:01,  2.87it/s]

## Sample

In [ ]:
from diffusion.sampler import sample
import matplotlib.pyplot as plt

# sample 64 images
samples = sample(ddpm, image_size=image_size, batch_size=64, channels=channels)

# show a random one
random_indexes = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50]

fig, axs = plt.subplots(1, 10, figsize=(20, 8))
for i in range(10):
    idx = random_indexes[i]
    axs[i].get_xaxis().set_visible(False)
    axs[i].get_yaxis().set_visible(False)
    axs[i].set_title(f"Image {idx}")
    axs[i].imshow(samples[-1][idx].reshape(image_size, image_size, channels), cmap="gray")
plt.show()

In [ ]:
import matplotlib.animation as animation

random_index = 5

fig = plt.figure()
ims = []
for i in range(timesteps):
    plt.title(f"T = {i}")
    im = plt.imshow(samples[i][random_index].reshape(image_size, image_size, channels), cmap="gray", animated=True)
    ims.append([im])

animate = animation.ArtistAnimation(fig, ims, interval=50, blit=True, repeat_delay=1000)
animate.save('diffusion64x64.gif')
plt.show()